## Make imports

In [1]:
import tensorflow as tf

In [2]:
# check if gpu is available
tf.config.experimental.list_physical_devices('GPU')
tf.debugging.set_log_device_placement(True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Simulation functions

In [626]:
def multiBrownian(M, N, dim, T, dtype=tf.float64):
    '''
    A multidimensional independent Brownian motion.
    M: Number of samples.
    N: Number of periods.
    dim: Dimension of the brownian motion.
    T: Time interval
    '''
    
    dt = tf.convert_to_tensor(T / (N-1), dtype=dtype)
    Z = tf.math.sqrt(dt) * tf.random.normal((M, N, dim), dtype=dtype)
    return tf.math.cumsum(Z, axis=1, exclusive=True)

def geometricBM(nb_samples, nb_periods, dim, T, S0, rate, div_yield, sigma, corr, dtype=tf.float64):
    '''
    This function will simulate a geometric BM
    
    S0: Initial value. shape = (dim)
    rate: Risk free interest rate (scalar).
    div_yield: Dividends yields. shape = (dim)
    sigma: Volatilities. shape = (dim)
    corr: Correlation matrix. shape = (dim, dim) 
    '''
    # convert to tensor
    S0 = tf.convert_to_tensor(S0, dtype=dtype)
    div_yield = tf.convert_to_tensor(div_yield, dtype=dtype)
    sigma = tf.convert_to_tensor(sigma, dtype=dtype)
    corr = tf.convert_to_tensor(corr, dtype=dtype)
        
    # time grid
    t = tf.range(0, T + T / nb_periods, T / (nb_periods - 1), dtype=dtype) 
    t = tf.reshape(t, [nb_periods, 1])
    
    # drift
    u = rate - div_yield - sigma ** 2 / 2
    u = tf.reshape(u, [1, dim])

    # get brownian motion
    BM = multiBrownian(nb_samples, nb_periods, dim, T)    
    
    if dim > 1:
        # covariance matrix -------------------
        #temp = sigma[None] * sigma[:, None]
        #cov = tf.multiply(temp, corr)
        #A = tf.linalg.cholesky(cov)

        # or
        sigma_ = tf.reshape(sigma, [dim, 1])
        A = tf.linalg.cholesky(corr)
        A = tf.multiply(A, sigma_)
        # -------------------------------------        
        diffusion_term = tf.linalg.matvec(A, BM)  
    else:
        diffusion_term = BM
        
    res = tf.math.exp(u*t + diffusion_term)    
    return S0 * res

## Test simulation

In [627]:
import matplotlib.pyplot as plt

nb_periods = 1000
dim = 2
T = 1
S0 = (1, 1)
corr = [[1.000000001, -1],
        [-1, 1.000000001]]
sigma = (0.2, 0.2)
div_yield = 0
rate = 0.2

x = geometricBM(1, nb_periods, dim, T, S0, rate, 0, sigma, corr)
x = tf.reshape(x, [nb_periods, dim]).numpy()
x = tf.transpose(x)

t = np.linspace(0, 1, N)
plt.plot(t, x[0])
plt.plot(t, x[1])

## Test martingality

In [628]:
# 10^6 samples with N = 10 and dim = 2 ==>> 2 * 10^7 doubles = 160 MB

nb_periods = 10
nb_samples = 1000000 #10^6
dim = 2
T = 1
S0 = (1, 2)
corr = [[1, 0],
        [0, 1]]
sigma = (0.2, 0.2)
div_yield = 0
rate = 0.02

x = geometricBM(nb_samples, nb_periods, dim, T, S0, rate, 0, sigma, corr)

In [629]:
tf.reduce_mean(x, axis=0)

<tf.Tensor: shape=(10, 2), dtype=float64, numpy=
array([[1.        , 2.        ],
       [1.00225597, 2.00450913],
       [1.00443044, 2.00885863],
       [1.00673376, 2.01340415],
       [1.0089534 , 2.01792163],
       [1.01118513, 2.02219168],
       [1.01346656, 2.02673954],
       [1.01581648, 2.03096394],
       [1.01799909, 2.03555852],
       [1.0202092 , 2.04007628]])>